<a id='07-nb'></a>

# Music Recommender Part 7: Clean Up

## Overview

----
### Clean up : Delete all Resources Created in the past 8 notebooks (nb 00-06)
In the past notebooks we have created many Amazon Resources; represented by their ARNs : Amazon Resource Names.
In order not to incur any cost in keeping those resources running, such as endpoints etc. We will use this notebook as a reminder to clean up and delete all the resources you have created in this music recommendation example.

First we will read in all parameters saved in the 'music-rec' namespace as we went from one notebook to the next,
second we will use a little utility under the `./code/demo_helpers.py` script file to actually delete all resources passed
----
### Contents
- [Overview](00_overview_arch_data.ipynb)
- [Part 1: Data Prep using Data Wrangler](01_music_dataprep.flow)
- [Part 2a: Feature Store Creation - Tracks](02a_export_fg_tracks.ipynb)
- [Part 2b: Feature Store Creation - User Preferences](02b_export_fg_5star_features.ipynb)
- [Part 2c: Feature Store Creation - Ratings](02c_export_fg_ratings.ipynb)
- [Part 3: Train Model with Debugger Hooks. Set Artifacts and Register Model.](03_train_model_lineage_registry_debugger.ipynb)
- [Part 4: Deploy Model & Inference using Online Feature Store](04_deploy_infer_explain.ipynb)
- [Part 5: Model Monitor](05_model_monitor.ipynb)
- [Part 6: SageMaker Pipelines](06_pipeline.ipynb)
- [Part 7: Resource Cleanup](07_clean_up.ipynb)

In [1]:
import json
import boto3
import pathlib
import sagemaker
import numpy as np
import pandas as pd
import awswrangler as wr

from sagemaker.estimator import Estimator
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import CreateModelStep
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.steps import ProcessingStep, TrainingStep
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.parameters import ParameterInteger, ParameterFloat, ParameterString
from sagemaker.feature_store.feature_group import FeatureGroup

ModuleNotFoundError: No module named 'awswrangler'

In [12]:
import sys
import pprint
sys.path.insert(1, './code')
from parameter_store import ParameterStore

ps = ParameterStore(verbose=False)

parameters = ps.read('music-rec')

bucket = parameters['bucket']
prefix = parameters['prefix']
ratings_data_source = parameters['ratings_data_source']
tracks_data_source = parameters['tracks_data_source']
val_data_uri = f"s3://{bucket}/{prefix}/data/val/val_data.csv"

pipeline_endpoint_name = parameters['pipeline_endpoint_name']
pipeline_name = parameters['pipeline_name']

fg_name_tracks = parameters['fg_name_tracks']
fg_name_ratings = parameters['fg_name_ratings']
fg_name_user_preferences = parameters['fg_name_user_preferences']

dw_ecrlist = parameters['dw_ecrlist']

pipeline_name = parameters['pipeline_name']
dataprep_pipeline_name = parameters['dataprep_pipeline_name']
train_deploy_pipeline_name = parameters['train_deploy_pipeline_name']

endpoint_name = parameters['endpoint_name']
pipeline_endpoint_name = parameters['pipeline_endpoint_name']

mpg_name = parameters['mpg_name']

In [13]:
region = boto3.Session().region_name
boto3.setup_default_session(region_name=region)
boto_session = boto3.Session(region_name=region)

s3_client = boto3.client('s3', region_name=region)

sagemaker_boto_client = boto_session.client('sagemaker')
sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client)
sagemaker_role = sagemaker.get_execution_role()

account_id = boto3.client('sts').get_caller_identity()["Account"]

In [14]:
# when demo_helpers.delete_project_resources() is ran it will delete all the resources created by this demo
sys.path.insert(1, './code')
import demo_helpers  # our custom set of functions


def remove_all_resources():
    demo_helpers.delete_project_resources(
        sagemaker_boto_client=sagemaker_boto_client, 
        sagemaker_session=sagemaker_session,
        endpoint_names=[pipeline_endpoint_name, endpoint_name],
        pipeline_names=[pipeline_name, dataprep_pipeline_name, train_deploy_pipeline_name], 
        mpg_name=mpg_name,
        feature_groups=[fg_name_ratings, fg_name_tracks, fg_name_user_preferences], 
        prefix=prefix,
        delete_s3_objects=True,
        bucket_name=bucket
    )

In [17]:
# Uncomment the next line and run to delete all resources
remove_all_resources()

Deleting artifact arn:aws:sagemaker:us-east-2:738335684114:artifact/2c625cb9a983d9fc346ab87d1e4feb63 None
Deleting artifact arn:aws:sagemaker:us-east-2:738335684114:artifact/699203e2eea79621c5e91be6ce0b0306 None
Deleting artifact arn:aws:sagemaker:us-east-2:738335684114:artifact/5488f0b22a88e6df902994d04b23f100 None
Deleting artifact arn:aws:sagemaker:us-east-2:738335684114:artifact/3ae904a337b89a8c194cc04d249c9398 None
Deleting artifact arn:aws:sagemaker:us-east-2:738335684114:artifact/8db02f01310b1533998dc42bca4f944d None
Deleting artifact arn:aws:sagemaker:us-east-2:738335684114:artifact/e5898eb08279fe0b3e32249848f6a992 None
Deleting artifact arn:aws:sagemaker:us-east-2:738335684114:artifact/5930f0b9c8ecc418ba6bb02022ff545c None
Deleting artifact arn:aws:sagemaker:us-east-2:738335684114:artifact/f01f2ce333829b11654591184285c203 None
Deleting artifact arn:aws:sagemaker:us-east-2:738335684114:artifact/ca31cc5fcd5602778b62ec47e732156e None
Deleting artifact arn:aws:sagemaker:us-east-2: